In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv(r'C:\Users\omarf\Downloads\train.csv')
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [4]:
test = pd.read_csv(r'C:\Users\omarf\Downloads\test.csv')
test.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5


In [5]:
submission = pd.read_csv(r'C:\Users\omarf\Downloads\sample_submission.csv')
submission.head()

,id,Calories
0,750000,88.283
1,750001,88.283
2,750002,88.283
3,750003,88.283
4,750004,88.283


In [6]:
train.info()
train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          750000 non-null  int64  
 1   Sex         750000 non-null  object 
 2   Age         750000 non-null  int64  
 3   Height      750000 non-null  float64
 4   Weight      750000 non-null  float64
 5   Duration    750000 non-null  float64
 6   Heart_Rate  750000 non-null  float64
 7   Body_Temp   750000 non-null  float64
 8   Calories    750000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 51.5+ MB


id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [7]:
# Feature engineering for train dataset

# 1. Encode categorical variables
train_fe = train.copy()
train_fe['Sex'] = train_fe['Sex'].map({'male': 0, 'female': 1})

# 2. Create BMI feature
train_fe['BMI'] = train_fe['Weight'] / ((train_fe['Height'] / 100) ** 2)

# 3. Interaction features
train_fe['Duration_HeartRate'] = train_fe['Duration'] * train_fe['Heart_Rate']
train_fe['Duration_BodyTemp'] = train_fe['Duration'] * train_fe['Body_Temp']

# 4. Drop 'id' column (not useful for prediction)
train_fe = train_fe.drop(columns=['id'])

# train_fe is now suitable for supervised ML algorithms like Linear Regression
train_fe.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,BMI,Duration_HeartRate,Duration_BodyTemp
0,0,36,189.0,82.0,26.0,101.0,41.0,150.0,22.955684,2626.0,1066.0
1,1,64,163.0,60.0,8.0,85.0,39.7,34.0,22.582709,680.0,317.6
2,1,51,161.0,64.0,7.0,84.0,39.8,29.0,24.690405,588.0,278.6
3,0,20,192.0,90.0,25.0,105.0,40.7,140.0,24.414062,2625.0,1017.5
4,1,38,166.0,61.0,25.0,102.0,40.6,146.0,22.136740,2550.0,1015.0


In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error

# Prepare features and target
X_dt = train_fe.drop(columns=['Calories'])
y_dt = train_fe['Calories']

# Train Decision Tree Regressor
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_dt, y_dt)

# Predict on training data
y_dt_pred = dt_model.predict(X_dt)

# Calculate RMSLE
dt_rmsle = np.sqrt(mean_squared_log_error(y_dt, y_dt_pred))
print(f"Decision Tree RMSLE on train set: {dt_rmsle:.6f}")

Decision Tree RMSLE on train set: 0.003928


In [12]:
from IPython.display import FileLink

# Create submission DataFrame using train dataset predictions
submission_train = pd.DataFrame({
    'id': train.index,
    'Calories': y_dt_pred
})

# Save to CSV
submission_train.to_csv('submission_train.csv', index=False)

# Provide a download link (for Jupyter Notebook)
FileLink('submission_train.csv')

c:\Users\omarf\OneDrive\Documents\submission_train.csv

In [13]:
assert submission.shape[0] == 250000, "Submission must have 250000 rows"

In [16]:
from IPython.display import FileLink

# Save the submission DataFrame to CSV
submission.to_csv('submission-Class.csv', index=False)

# Provide a download link
FileLink('submission-Class.csv')

c:\Users\omarf\OneDrive\Documents\submission-Class.csv